In [1]:
import pyspark
import os

os.environ['SPARK_DIST_CLASSPATH']="""/opt/spark2/conf/yarn-conf:/opt/hadoop2/share/hadoop/common/lib/*:/opt/hadoop2/share/hadoop/common/*:/opt/hadoop2/share/hadoop/hdfs:/opt/hadoop2/share/hadoop/hdfs/lib/*:/opt/hadoop2/share/hadoop/hdfs/*:/opt/hadoop2/share/hadoop/yarn/lib/*:/opt/hadoop2/share/hadoop/yarn/*:/opt/hadoop2/share/hadoop/mapreduce/lib/*:/opt/hadoop2/share/hadoop/mapreduce/*"""


from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

from pyspark.streaming import StreamingContext

# База

In [2]:
# нужно не меньше 2х ядер: 1 - для получения данных, 1 - для процессинга
conf = SparkConf().set("spark://cnt-cls-m1:7070","local[2]") 
sc = SparkContext(master='local[2]')
sc

<SparkContext master=local[2] appName=studtask>

In [3]:
from pyspark.streaming import StreamingContext

#обект стриминга и длительность батча (сек)
ssc = StreamingContext(sparkContext=sc,batchDuration=10) 
ssc

In [7]:
# определяем сокет для получения данных и порт
dstream1 = ssc.socketTextStream("localhost", 9999)
dstream1
dstream1.pprint()

In [8]:
ssc.start()

-------------------------------------------
Time: 2020-09-10 10:25:50
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:26:00
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:26:10
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:26:20
-------------------------------------------
asdasda 
asdas ada sda asda sda da asdas

-------------------------------------------
Time: 2020-09-10 10:26:30
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:26:40
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:26:50
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:27:00
-------------------------------------------

---------------------------------------

In [9]:
ssc.stop(False,False)

# Функции над стриминг процессом

In [10]:
ssc = StreamingContext(sparkContext=sc,batchDuration=20)
ssc

In [11]:
ds1 = ssc.socketTextStream(hostname = "localhost", port = 9999)
print(ds1)


In [12]:
# посчитаем повторяющиеся слова
ds2 = ds1#.flatMap(lambda x:x.split(" ")).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
print(ds2)
print(ds2.pprint())

None


In [13]:
ssc.start()

-------------------------------------------
Time: 2020-09-10 10:33:20
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:33:40
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:34:00
-------------------------------------------
('', 209)
('1', 1)
('mc', 7)
('su', 11)
('4', 1)
('echo', 1)
('ls', 4)
('8', 1)
('9', 1)
('mkdir', 1)
...

-------------------------------------------
Time: 2020-09-10 10:34:20
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:34:40
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:35:00
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:35:20
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:35:40
-------------------

In [ ]:
# сохранить результат локально 
ds2.saveAsTextFiles("file:///home/{}/{}".format("", ""))

In [14]:
# выбор типа остановка, можно сделать стоп и спарк контекста или стриминг контенкста
ssc.stop(stopSparkContext=False,stopGraceFully=False) 

# parallelize

In [15]:
ssc = StreamingContext(sparkContext=sc,batchDuration=20)
ssc

In [17]:
queueOfRDDs = []
for i in range(10):
    queueOfRDDs.append(sc.parallelize([i]))
print(queueOfRDDs)    

[ParallelCollectionRDD[106] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[107] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[108] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[109] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[110] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[111] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[112] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[113] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[114] at parallelize at PythonRDD.scala:195, ParallelCollectionRDD[115] at parallelize at PythonRDD.scala:195]


In [22]:
# создадим Dstream из очереди из RDD
dstream_rdd = ssc.queueStream(queueOfRDDs).persist(pyspark.StorageLevel.MEMORY_AND_DISK)
dstream_rdd.pprint()

In [24]:
# путь указывается до директории (файловая система - hadoop)
#dstream_file = ssc.textFileStream("file:///home/{}/{}".format("", ""))
#dstream_file.pprint()

In [25]:
ssc.start()

-------------------------------------------
Time: 2020-09-10 10:43:40
-------------------------------------------
0

-------------------------------------------
Time: 2020-09-10 10:43:40
-------------------------------------------
0

-------------------------------------------
Time: 2020-09-10 10:43:40
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:44:00
-------------------------------------------
1

-------------------------------------------
Time: 2020-09-10 10:44:00
-------------------------------------------
1

-------------------------------------------
Time: 2020-09-10 10:44:00
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 10:44:20
-------------------------------------------
2

-------------------------------------------
Time: 2020-09-10 10:44:20
-------------------------------------------
2

-------------------------------------------
Time: 2020-09-10 10:44:2

In [26]:
ssc.stop(stopSparkContext=False)

# SQL | DF

In [27]:
ssc = StreamingContext(sparkContext=sc,batchDuration=20)
ssc

In [28]:
# инициализация нужного контекста
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)
sqlc

In [31]:
# создаем свой Df
d1 = (
        sqlc
         .createDataFrame(
             [(1,2300,1),(2,3000,1),(1,1000,2),
              (2,1500,2),(2,4200,3),(3,2000,1),
              (2,500,4),(3,2000,2)],
              ["id","sales","month"]
         )
     )
d1.show()

+---+-----+-----+
| id|sales|month|
+---+-----+-----+
|  1| 2300|    1|
|  2| 3000|    1|
|  1| 1000|    2|
|  2| 1500|    2|
|  2| 4200|    3|
|  3| 2000|    1|
|  2|  500|    4|
|  3| 2000|    2|
+---+-----+-----+



In [32]:
# добавим возможность обращаться SQL
d1.createOrReplaceTempView("t1")

# сделаем расчет среднего кол-ва продаж на каждый месяц

(
    sqlc
        .sql("""
             select id
                    , sales
                    , month
                    , avg(sales) over(partition by id order by month) as avgsale
             from t1""")
).show()

+---+-----+-----+-------+
| id|sales|month|avgsale|
+---+-----+-----+-------+
|  1| 2300|    1| 2300.0|
|  1| 1000|    2| 1650.0|
|  3| 2000|    1| 2000.0|
|  3| 2000|    2| 2000.0|
|  2| 3000|    1| 3000.0|
|  2| 1500|    2| 2250.0|
|  2| 4200|    3| 2900.0|
|  2|  500|    4| 2300.0|
+---+-----+-----+-------+



In [33]:
ssc = StreamingContext(sparkContext=sc,batchDuration=20)
ssc

In [34]:
dstream1 = ssc.socketTextStream("localhost",9999)
dstream2 = dstream1.window(windowDuration=60)

In [35]:
dstream3 = dstream2.flatMap(lambda x:x.split(" ")).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
dstream3.pprint()

In [36]:
ssc.start()

-------------------------------------------
Time: 2020-09-10 10:54:00
-------------------------------------------
('\x1b[A\x1b[A\x1b[B\x1b[B\\', 1)
('ssc.stop(stopSparkContext=False)', 1)
('sda', 1)

-------------------------------------------
Time: 2020-09-10 10:54:20
-------------------------------------------
('\x1b[A\x1b[A\x1b[B\x1b[B\\', 1)
('ssc.stop(stopSparkContext=False)', 1)
('sda', 1)

-------------------------------------------
Time: 2020-09-10 10:54:40
-------------------------------------------
('\x1b[A\x1b[A\x1b[B\x1b[B\\', 1)
('', 33)
('1|', 5)
('|', 7)
('4200|', 1)
('3|', 3)
('ssc.stop(stopSparkContext=False)', 1)
('sda', 1)
('2300|', 1)
('2|', 7)
...

-------------------------------------------
Time: 2020-09-10 10:55:00
-------------------------------------------
('', 33)
('1|', 5)
('|', 7)
('4200|', 1)
('3|', 3)
('2300|', 1)
('2|', 7)
('3000|', 1)
('1000|', 1)
('1500|', 1)
...

-------------------------------------------
Time: 2020-09-10 10:55:20
--------------------

In [37]:
ssc.stop(False)

In [42]:
#SQL

In [51]:
ssc = StreamingContext(sparkContext=sc,batchDuration=15)
ssc

In [53]:
# создать DF из RDD 
# Нужна операция, которая будет приминяться к RDD в Dstream

def f(rdd):
    rdd2 = rdd.flatMap(lambda x:x.split(" ")).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
    df = rdd2.toDF(["word","count"])
    df.show()
    #df.write.format("csv").save("file:///home/{}/{}")


ds1 = ssc.socketTextStream("localhost",port = 9999)
ds2 = ds1.foreachRDD(f)
ds2

In [54]:
ssc.start()

In [55]:
ssc.stop(False)

+--------------------+-----+
|                word|count|
+--------------------+-----+
|               18471|    3|
|                    |  453|
|                 0.0|   36|
|                 0.2|    6|
|             1439380|    3|
|               66576|    3|
|                   ?|   24|
|                 Ssl|   24|
|               Sep08|   24|
|                0:11|   12|
|    /usr/bin/python3|   21|
|                  -m|   21|
|  ipykernel_launcher|   21|
|                  -f|   21|
|/tmp/ezadru/.loca...|    3|
|                root|   27|
|               18573|    3|
|             1439636|    3|
|               65232|    3|
|/tmp/ezadru/.loca...|    3|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                    |  234|
|                 0.0|   20|
|                 0.2|    2|
|                 Ssl|   10|
|               Sep08|   10|
|                  -f|   10|
|               6

# Tweeter stream

In [4]:
import twitter
api_auth = twitter.Api(consumer_key='NFdepyaFSJiEcLZr72QlEeejE',
consumer_secret='63BYnnRZopkEPw8YH28MrE28WEY3B89hgyoMdQWjPfRtqiQhfD',
access_token_key='976856153912201228-fu5upI5jkml1QytOe4nupFSmCaT4V7k',
access_token_secret='jyg4njgsexZ2jJM4TBFhyBzBYCA14VEoU9GgWF0T6zkGw')

In [58]:
print(api_auth.VerifyCredentials())

{"created_at": "Thu Mar 22 16:20:31 +0000 2018", "default_profile": true, "default_profile_image": true, "followers_count": 3, "id": 976856153912201228, "id_str": "976856153912201228", "name": "karnakar", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "karnaTrainer"}


In [63]:
print(api_auth.GetSearch("modi"))

[Status(ID=1303548439217672192, ScreenName=pbhushan1, Created=Wed Sep 09 04:19:01 +0000 2020, Text='People involved from the beginning with the Ayodhya temple movement accuse the BJP of swallowing 1400 Crores collec… https://t.co/BEuKwJXWEI'), Status(ID=1303906228179841024, ScreenName=RahulGandhi, Created=Thu Sep 10 04:00:44 +0000 2020, Text='The policies of Modi Govt have caused the loss of crores of jobs and a historic fall in GDP. \n\nIt has crushed the f… https://t.co/qaqiO9jpG4'), Status(ID=1303573874148163587, ScreenName=FrustIndian, Created=Wed Sep 09 06:00:05 +0000 2020, Text="You call Modi fascist, compare India with Hitler's Germany nothing happens to you. You question Congress Shiv Sena,… https://t.co/VOTdDrpiCX"), Status(ID=1304005375637684224, ScreenName=AwaraSazde, Created=Thu Sep 10 10:34:43 +0000 2020, Text='RT @Rakesh_Bassi7: #SpeakUpForJobs \nmodi ji decreasing number of in public sectors since 2014.\njust have a look in this list and see the nu…'), Status(ID=13040053

In [15]:
ssc = StreamingContext(sparkContext=sc,batchDuration=15)
ssc

In [87]:
import json
import time
#Load the tweets into a directory for every 20 seconds and load all the tweets as a file

fileno = 0
while fileno < 3:  #:(1):
    file = open("/home/aselezn/tweeterdata/tweets_{}".format(fileno),"w")
    tweets = api_auth.GetSearch("modi")
    for tweet in tweets:    
        tweet_json = json.loads((str)(tweet))
        print(tweet)
        file.write((str)(tweet_json))
        file.write("\n")
    fileno = fileno+1 
    time.sleep(20)

{"created_at": "Wed Sep 09 04:19:01 +0000 2020", "favorite_count": 9745, "hashtags": [], "id": 1303548439217672192, "id_str": "1303548439217672192", "lang": "en", "retweet_count": 3925, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "text": "People involved from the beginning with the Ayodhya temple movement accuse the BJP of swallowing 1400 Crores collec\u2026 https://t.co/BEuKwJXWEI", "truncated": true, "urls": [{"expanded_url": "https://twitter.com/i/web/status/1303548439217672192", "url": "https://t.co/BEuKwJXWEI"}], "user": {"created_at": "Fri Jan 10 09:18:00 +0000 2014", "default_profile": true, "description": "Public Interest Lawyer and Activist; \nDo and say what is just fair and in public interest, regardless of immediate fallout", "favourites_count": 67, "followers_count": 1804097, "friends_count": 50, "id": 2284802828, "id_str": "2284802828", "location": "New Delhi, India", "name": "Prashant Bhushan", "profile_background

{"created_at": "Wed Sep 09 04:19:01 +0000 2020", "favorite_count": 9745, "hashtags": [], "id": 1303548439217672192, "id_str": "1303548439217672192", "lang": "en", "retweet_count": 3925, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "text": "People involved from the beginning with the Ayodhya temple movement accuse the BJP of swallowing 1400 Crores collec\u2026 https://t.co/BEuKwJXWEI", "truncated": true, "urls": [{"expanded_url": "https://twitter.com/i/web/status/1303548439217672192", "url": "https://t.co/BEuKwJXWEI"}], "user": {"created_at": "Fri Jan 10 09:18:00 +0000 2014", "default_profile": true, "description": "Public Interest Lawyer and Activist; \nDo and say what is just fair and in public interest, regardless of immediate fallout", "favourites_count": 67, "followers_count": 1804097, "friends_count": 50, "id": 2284802828, "id_str": "2284802828", "location": "New Delhi, India", "name": "Prashant Bhushan", "profile_background

{"created_at": "Wed Sep 09 04:19:01 +0000 2020", "favorite_count": 9746, "hashtags": [], "id": 1303548439217672192, "id_str": "1303548439217672192", "lang": "en", "retweet_count": 3925, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "text": "People involved from the beginning with the Ayodhya temple movement accuse the BJP of swallowing 1400 Crores collec\u2026 https://t.co/BEuKwJXWEI", "truncated": true, "urls": [{"expanded_url": "https://twitter.com/i/web/status/1303548439217672192", "url": "https://t.co/BEuKwJXWEI"}], "user": {"created_at": "Fri Jan 10 09:18:00 +0000 2014", "default_profile": true, "description": "Public Interest Lawyer and Activist; \nDo and say what is just fair and in public interest, regardless of immediate fallout", "favourites_count": 67, "followers_count": 1804098, "friends_count": 50, "id": 2284802828, "id_str": "2284802828", "location": "New Delhi, India", "name": "Prashant Bhushan", "profile_background

In [16]:
twitter_stream = ssc.textFileStream("/home/aselezn/tweeterdata/")

def f1(rdd):
    rdd.count()
    
    
twitter_stream.foreachRDD(f1)

In [17]:
twitter_stream.pprint()

In [18]:
ssc.start()

-------------------------------------------
Time: 2020-09-10 12:50:45
-------------------------------------------

-------------------------------------------
Time: 2020-09-10 12:51:00
-------------------------------------------



In [19]:
ssc.stop(False)